Service Mesh mit Istio (Infrastruktur)
==============================

Cloud-Plattformen bieten den Unternehmen, die sie verwenden, zahlreiche Vorteile.

Dazu müssen Entwickler das Architekturmuster Microservices verwenden, was dazu führt das schnell Unterschiedliche Versionen von Microservices parallel betrieben werden müssen.

Mit den vielfältigen Funktionen von [Istio](https://istio.io/) kann man eine verteilte Microservice-Architektur erfolgreich und effizient ausführen und auf einheitliche Weise Microservices absichern, verbinden und überwachen.

Es ist ein vollständiger Open-Source-Service-Mesh, das auf vorhandenen verteilten Anwendungen aufbaut.

Istio wurde bereits mittels dem Script [istio.sh](https://raw.githubusercontent.com/mc-b/lerncloud/main/services/istio.sh) installiert.

Microservice Istio Sample
-------------------------------------

Das [Microservice Istio Sample](https://github.com/ewolff/microservice-istio) von 
Eberhard Wolff, ist eine vereinfachte Variante des Messaging Beispiels und gibt einen Einblick in Istio.

Es kann wie folgt gestartet werden:


In [ ]:
! kubectl label namespace default istio-injection=enabled
! kubectl apply -f https://raw.githubusercontent.com/ewolff/microservice-istio/master/microservice-istio-demo/infrastructure-dockerhub.yaml
! kubectl apply -f https://raw.githubusercontent.com/ewolff/microservice-istio/master/microservice-istio-demo/microservices-native-dockerhub.yaml

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port (port-based-routing) als URL aufbereiten.

Wird dieser URL in einem neuen Tab geöffnet, wird ein Menu angezeigt.

In [ ]:
%%bash
[ ! -f ~/work/server-ip ] && { echo "replace-with-server-ip" >~/work/server-ip; }
PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
echo http://$(cat ~/work/server-ip):$PORT/

- - - 

Tools
=====

Kiali — Observability
------------------------------
[Kiali]() ist eine Web UI für Istio. Es hilft Ihnen, die Struktur Ihres Servicenetzes und deren Topologie zu verstehen.

Jaeger — Tracing
-------------------------

Die Ablaufinformationen für Kiali werden durch das verteiltes Tracing-System [Jaeger]() bereitgestellt.

Beide Tools sind jedoch nur auf dem Cluster zugänglich. Deshalb müssen wir diese zuerst freischalten bzw. gegen NodePort öffnen:

In [ ]:
! kubectl get service -n istio-system -l app=kiali  -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! kubectl get service -n istio-system -l app=jaeger -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -

In [ ]:
! echo "Kiali   UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=kiali -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')
! echo "Jaeger  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service/tracing -o jsonpath='{.spec.ports[?(@.name=="http-query")].nodePort}')

Prometheus
----------

Istio enthält eine Installation von [Prometheus](https://prometheus.io/). 

Dieses Werkzeug sammelt Metriken von den Proxies ein, über welche die Microservices miteinander kommunizieren. Es stellt daher die Basis für
das Monitoring dar. 

In [ ]:
! kubectl get service -n istio-system -l app=prometheus -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Prometheus UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=prometheus -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')

Grafana
-------

Prometheus bietet nur sehr limitierte Dashboards an. Deswegen hat Istio außerdem eine Installation von [Grafana](https://grafana.com/), das viel bessere Graphen und
Dashboards anbietet.

In [ ]:
! kubectl get -n istio-system service/grafana -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Grafana UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service/grafana -o=jsonpath='{ .spec.ports[0].nodePort }')

Metriken sind nur sinnvoll, wenn das System unter Last ist. Das nachfolgende Shell-Skript nutzt curl, um Shipping und Order 1.000 mal aufzurufen.

In [ ]:
%%bash
[ ! -f ~/work/server-ip ] && { echo "replace-with-server-ip" >~/work/server-ip; }
PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')

for i in `seq 1 1000`;
do
    curl -s -o /dev/null -I -w "%{http_code}" http://shipping
    curl -s -o /dev/null -I -w "%{http_code}" http://order
    curl -s -o /dev/null -I -w "%{http_code}" http://invoicing    
    echo
done